In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")
datset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("comments")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [3]:
table_ref = dataset_ref.table("stories")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,id,by,score,time,time_ts,title,url,text,deleted,dead,descendants,author
0,6940813,sarath237,0,1387536270,2013-12-20 10:44:30+00:00,Sheryl Brindo Hot Pics,http://www.youtube.com/watch?v=ym1cyxneB0Y,Sheryl Brindo Hot Pics,None,True,NaN,sarath237
1,6991401,123123321321,0,1388508751,2013-12-31 16:52:31+00:00,Are you people also put off by the culture of ...,,They&#x27;re pretty explicitly &#x27;startup f...,None,True,NaN,123123321321
2,1531556,ssn,0,1279617234,2010-07-20 09:13:54+00:00,New UI for Google Image Search,http://googlesystem.blogspot.com/2010/07/googl...,Again following on Bing's lead.,None,None,0.0,ssn
3,5012398,hoju,0,1357387877,2013-01-05 12:11:17+00:00,Historic website screenshots,http://webscraping.com/blog/Generate-website-s...,Python script to generate historic screenshots...,None,None,0.0,hoju
4,7214182,kogir,0,1401561740,2014-05-31 18:42:20+00:00,Placeholder,,Mind the gap.,None,None,0.0,kogir


In [4]:
join_query = """
            WITH c AS
            (
            SELECT parent, COUNT(*) as num_comments
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            )
            SELECT s.id as story_id, s.by, s.title, c.num_comments
            FROM `bigquery-public-data.hacker_news.stories` AS s
            LEFT JOIN c
            ON s.id = c.parent
            WHERE EXTRACT(DATE FROM s.time_ts) = '2012-01-01'
            ORDER BY c.num_comments DESC
            """

In [5]:
join_result = client.query(join_query).result().to_dataframe()
join_result.head()

,story_id,by,title,num_comments
0,3412900,whoishiring,Ask HN: Who is Hiring? (January 2012),154.0
1,3412901,whoishiring,Ask HN: Freelancer? Seeking freelancer? (Janua...,97.0
2,3412643,jemeshsu,Avoid Apress,30.0
3,3412891,Brajeshwar,"There's no shame in code that is simply ""good ...",27.0
4,3414012,ramanujam,Impress.js - a Prezi like implementation using...,27.0


In [6]:
join_result.tail()

,story_id,by,title,num_comments
439,3413074,xxqs,"New age telephony is here, yet undervalued",NaN
440,3413482,FluidDjango,The Internet Is People,NaN
441,3412848,jamespcole2,"Living in the Cloud, some useful tips",NaN
442,3413953,powerbase,Play Zelda: Skyward Sword in Ubuntu.,NaN
443,3413683,zem,San Francisco minimum wage crosses $10,NaN


In [7]:
union_query = """
            SELECT c.by
            FROM `bigquery-public-data.hacker_news.comments` AS c
            WHERE EXTRACT(DATE FROM c.time_ts) = '2014-01-01'
            UNION DISTINCT
            SELECT s.by
            FROM `bigquery-public-data.hacker_news.stories` AS s
            WHERE EXTRACT(DATE FROM s.time_ts) = '2014-01-01'
            """

In [8]:
union_result = client.query(union_query).result().to_dataframe()
union_result.head()

,by
0,alenn34
1,projuce
2,alifaizan
3,hackhackhack
4,lelf


In [9]:
len(union_result)

2282

## Exercise

In [10]:
dataset_ref = client.dataset("stackoverflow", project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("posts_questions")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,34307193,Reuse of WCF Binding class instances,<p>I create NetTcpBinding-s on application's c...,NaN,1,0,None,2015-12-16 08:32:50.587000+00:00,NaN,2015-12-20 23:32:52.530000+00:00,NaT,None,NaN,None,5152658,None,1,1,c#|wcf|wcf-binding|channelfactory|webchannelfa...,256
1,34333301,Upgrade to Free jqGrid 4.11 - inline edit with...,<p>The grid defines the field as:</p>\n\n<pre>...,34335032.0,1,0,None,2015-12-17 11:20:19.803000+00:00,1.0,2015-12-18 10:23:35.380000+00:00,2015-12-18 10:23:35.380000+00:00,None,470383.0,None,470383,None,1,2,jquery|jqgrid,256
2,34342966,Obscuring Default Plaintext password in Powers...,<p>I am working on a PowerCLI script that depl...,34343477.0,1,0,None,2015-12-17 19:43:01.430000+00:00,NaN,2015-12-17 20:15:31.240000+00:00,NaT,None,NaN,None,2373838,None,1,1,passwords|powercli,256
3,34349579,If/Else TextView actions iOS,<p>I have a <code>textview</code> in my <code>...,NaN,6,0,None,2015-12-18 06:20:09.150000+00:00,NaN,2015-12-18 07:26:54.473000+00:00,2015-12-18 06:44:37.900000+00:00,None,1508398.0,None,5588068,None,1,1,ios|objective-c,256
4,34373960,SpriteKit physics bodies not colliding when co...,<p>I am evaluating the iOS SpriteKit physics e...,NaN,1,0,None,2015-12-19 18:26:28.850000+00:00,1.0,2015-12-21 10:35:25.100000+00:00,2015-12-19 19:08:02.887000+00:00,None,1240557.0,None,1157345,None,1,3,ios|collision|skphysicsbody|skscene,256


In [11]:
table_ref = dataset_ref.table("posts_answers")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,20961156,None,"<p>I figured it out on my own, I think:</p>\n\...",None,None,0,None,2014-01-06 23:22:13.043000+00:00,None,2014-01-06 23:31:28.093000+00:00,2014-01-06 23:31:28.093000+00:00,None,2438423.0,None,2438423,20940483,2,0,None,None
1,20961162,None,<pre><code>WHERE post_content LIKE 'www.exampl...,None,None,0,None,2014-01-06 23:22:55.477000+00:00,None,2014-01-06 23:22:55.477000+00:00,NaT,None,NaN,None,3165602,20959992,2,0,None,None
2,20961163,None,"<pre><code>if (choice1 === ""rock"") {\n if (...",None,None,0,None,2014-01-06 23:22:59.443000+00:00,None,2014-01-06 23:22:59.443000+00:00,NaT,None,NaN,None,57135,20961119,2,0,None,None
3,20961171,None,"<p>Your ""brute force"" search is usually called...",None,None,0,None,2014-01-06 23:23:48.903000+00:00,None,2014-01-06 23:23:48.903000+00:00,NaT,None,NaN,None,91757,20960751,2,0,None,None
4,20961197,None,"<p>I would take a look at either <a href=""http...",None,None,0,None,2014-01-06 23:26:39.027000+00:00,None,2014-01-06 23:26:39.027000+00:00,NaT,None,NaN,None,2027567,20961131,2,0,None,None


In [12]:
first_query = """
            SELECT q.id AS q_id,
                MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) AS time_to_answer
            FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
            INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
            ON q.id = a.parent_id
            WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
            GROUP BY q_id
            ORDER BY time_to_answer
            """

In [13]:
first_result = client.query(first_query).result().to_dataframe()
print("Percentage of answered questions: %s%%" % \
     (sum(first_result["time_to_answer"].notnull())/len(first_result) *100))
print("Number of questions:", len(first_result))
first_result.head()

Percentage of answered questions: 100.0%
Number of questions: 134227


,q_id,time_to_answer
0,48186806,0
1,48199809,0
2,48316551,0
3,48320430,0
4,48359203,0


In [14]:
correct_query = """
            SELECT q.id AS q_id,
                MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) AS time_to_answer
            FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
            ON q.id = a.parent_id
            WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
            GROUP BY q_id
            ORDER BY time_to_answer
            """

In [16]:
correct_result = client.query(correct_query).result().to_dataframe()
print("Percentage of answered questions: %s%%" % \
     (sum(correct_result["time_to_answer"].notnull())/len(correct_result) *100))
print("Number of questions:", len(correct_result))
correct_result.head()

Percentage of answered questions: 82.4009331164247%
Number of questions: 162895


,q_id,time_to_answer
0,48117510,NaN
1,48070109,NaN
2,48180807,NaN
3,48099822,NaN
4,48268890,NaN


In [17]:
q_and_a_query = """
                SELECT q.owner_user_id AS owner_user_id,
                    MIN(q.creation_date) AS q_creation_date,
                    MIN(a.creation_date) AS a_creation_date
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                ON q.owner_user_id = a.owner_user_id 
                WHERE q.creation_date >= '2019-01-01' AND q.creation_date < '2019-02-01' 
                    AND a.creation_date >= '2019-01-01' AND a.creation_date < '2019-02-01'
                GROUP BY owner_user_id
                """

In [18]:
## check the users table
table_ref = dataset_ref.table("users")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,898,Gerhard Dinhof,<p>Passionate about Blockchain technology. Wor...,None,2008-08-10 08:59:07.857000+00:00,2019-10-31 09:07:00.130000+00:00,"Johannesburg, South Africa",1280,165,1,230,None,https://at.linkedin.com/in/gerharddinhof
1,963,Vesa Nieminen,<p>Main languages:<ul><li>Basic 1993-1995</li>...,None,2008-08-11 07:21:43.670000+00:00,2020-02-27 07:27:00.613000+00:00,Finland,151,16,0,165,None,http://vesanieminen.com
2,1196,aku,<p>I am an avid full-stack software developer/...,None,2008-08-13 12:33:04.803000+00:00,2020-02-22 18:59:14.650000+00:00,"Moscow, Russia",108832,302,50,4903,https://i.stack.imgur.com/iQcva.jpg?s=128&g=1,None
3,1456,Robin Minto,<p>I'm a developer/technical architect working...,None,2008-08-15 17:03:51.403000+00:00,2020-02-28 09:14:00.977000+00:00,"London, United Kingdom",12055,91,7,659,None,http://robinminto.com
4,1798,bill d,<p>Your basic code slinger.</p>,None,2008-08-18 15:49:32.283000+00:00,2017-05-26 18:18:26.217000+00:00,New Hampshire,618,13,0,64,None,http://about:blank


In [24]:
jan_user_query= """
            SELECT id, EXTRACT(DATE FROM creation_date) AS join_date
            FROM `bigquery-public-data.stackoverflow.users` 
            WHERE EXTRACT(DATE FROM creation_date) >= '2019-01-01' and 
                EXTRACT(DATE FROM creation_date) <= '2019-01-31'
            """
tmp_result = client.query(jan_user_query).result().to_dataframe()
tmp_result.head()

,id,join_date
0,10853644,2019-01-01
1,10853649,2019-01-01
2,10853778,2019-01-01
3,10853890,2019-01-01
4,10854166,2019-01-01


In [25]:
len(tmp_result)

142450

In [27]:
tmp_result.tail()

,id,join_date
142445,10998100,2019-01-31
142446,10998153,2019-01-31
142447,10998324,2019-01-31
142448,10998475,2019-01-31
142449,10998539,2019-01-31


In [30]:
jan_user_query= """
            WITH j AS 
            {
            SELECT id
            FROM `bigquery-public-data.stackoverflow.users`
            WHERE EXTRACT(DATE FROM creation_date) >= '2019-01-01' and 
                EXTRACT(DATE FROM creation_date) <= '2019-01-31'
            }
            LEFT JOIN `bigquery-public-data.stackoverflow.posts_questions` as q
            ON jan.id = q.id
            GROUP BY 
            """
tmp_result2 = client.query(jan_user_query).result().to_dataframe()
tmp_result2.head()

,id,join_date
0,10853644,2019-01-01
1,10853649,2019-01-01
2,10853778,2019-01-01
3,10853890,2019-01-01
4,10854166,2019-01-01


In [31]:
len(tmp_result2)

142450